In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
image_path = '/Users/patricknaylor/Desktop/Field_Detection/Images/Masked/'
torch.__version__


'2.0.0.dev20221209'

In [9]:

images_paths = glob.glob(image_path + '*.jpg')
mask_paths = glob.glob(image_path + '*.npy')
images = []
masks = []
for (img, msk) in zip(images_paths, mask_paths):
    images.append(cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB))
    masks.append(np.load(msk).T)


In [10]:
images_arr = np.stack(images)
masks_arr = np.stack(masks)
masks_arr = np.expand_dims(masks_arr[:, 25: -25, 25: -25], axis=3)
data_arr = np.concatenate((images_arr, masks_arr), axis=3)

In [11]:
images_arr = images_arr.astype(np.uint8)
masks_arr = masks_arr.astype(np.uint8)

In [19]:
class imageDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        mask = self.y[index]
        y = torch.tensor(mask)
        X = self.transform(image)
        return [X, y]

    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()])

    


    

In [22]:
print(np.shape(masks_arr))

(2753, 720, 1280, 1)


In [23]:
batch_size = 64
transformed_dataset = imageDataset(X=images_arr, y=masks_arr)
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [2000, 753])
train_dl = DataLoader(train_set, batch_size, shuffle=True, pin_memory=True)
val_dl = DataLoader(val_set, batch_size, shuffle=True, pin_memory=True)


In [25]:
train_iter = iter(train_dl)
val_iter = iter(val_dl)
images, masks = next(train_iter)